# `nx-cugraph`: Accelerating graph analysis on networkX 

In this notebook, we will show you how to use `nx-cugraph`, a collaboration between cuGraph and NetworkX, to accelerate graph analytics for NetworkX users.

Recently, NetworkX introduced a dispatcher that allows users to dispatch their workload to different backends. `nx-cugraph` is one of these backends, and uses GPU compute resources provided by `cugraph` to greatly improve performance.

The main requirements for using `nx-cugraph` are 
- Python versions 3.9, 3.10, or 3.11
- The latest development version of NetworkX
- cuGraph version 23.10 (nightly) 

Follow the instructions from `INSTALL.rst` on the NetworkX repository for instructions on installing the development version of `networkx`, which is `networkx-3.2rc0-dev0`.

In [ ]:
!pip show networkx

In [ ]:
!pip show cugraph

## Installing the latest development version of networkx

The following cells will
- uninstall the current version of networkx 
- clone the networkx repository one directory below the current working directory
- upgrade networkx to be the current development version with `pip install`

You may need to restart the kernel to have the development version of networkX be visible.

In [ ]:
!pip uninstall --yes networkx

In [ ]:
# Skip if current networkx repository already exists
!(cd ..; \
git clone https://github.com/networkx/networkx.git)

In [ ]:
!(cd ../networkx; \
pip install --upgrade --target=/opt/conda/envs/rapids/lib/python3.10/site-packages -e .[default])
# below: old pip install command,
# pip install --target=/opt/conda/envs/rapids/lib/python3.10/site-packages -e .[default])

In [ ]:
!pip show networkx

## Using networkx

In [ ]:
import networkx as nx
import pandas as pd
# from cugraph.datasets import hollywood

In [ ]:
nx.__version__, pd.__version__

The 'email-Eu-core' dataset, made up of anonymized email data from a large European institution, is a directed graph with 25571 edges and 1005 nodes. The following block reads this dataset into a networkx.Graph object. We then measure the performance of the call to betweenness centrality with an approximation of k=50 node samples, about 5% of the nodes.

In [ ]:
df = pd.read_csv("datasets/email-Eu-core.csv", header=None, names=['src', 'dst', 'wgt'], sep=' ')
Gnx = nx.from_pandas_edgelist(df, source='src', target='dst', create_using=nx.Graph)

In [ ]:
# Use instead of above cell if benchmark datasets are added
Gnx = hollywood.get_graph(create_using=nx.Graph)

In [ ]:
%%timeit
bc = nx.betweenness_centrality(Gnx, k=50)

Running betweenness_centrality using native networkX on the email-Eu-core dataset took around XXX. You, as the user, can choose the backend as an optional argument, providing you've installing your specific library, as seen below. A dispatchable networkX algorithm will use the chosen dispatcher to convert networkx Graph to the backend equivalent, dispatch the work, then convert back to the required nx form.

In [ ]:
import nx_cugraph as nxcg

In [ ]:
nx.__version__, pd.__version__, nxcg.__version__

In [ ]:
%%timeit
bc = nx.betweenness_centrality(Gnx, k=50, backend='cugraph')

## Customization

You can also use certain environment variable to control your backend choice, fallback settings, and testing options
- NETWORKX_TEST_BACKEND - (was previously NETWORKX_GRAPH_CONVERT) 
- NETWORKX_FALLBACK_TO_NX - if True, then networkx will be used if an algorithm isn't supported by the chosen backend
- NETWORKX_AUTOMATIC_BACKENDS - Enables automatic conversions to backends

This approach is easy to use, since you change only the algorithmic calls in your workload without restructuring the rest of your work. You can specify when and where to use the backend this way. While an optional argument is convenient, there might be scenarios where you don't want to edit your script or pipeline. Fortunately, the dispatcher allows for environment variables to configure your backend, fallback, and acceleration preferences.

### Utilities

The `nx-cugraph` package allows conversion from networkx Graph objects to cugraph-compatible Graph objects. They are distinct from cugraph Graph objects and are more limited, but as the backend grows, so will its capabilities.

In [ ]:
%%timeit
Gcg = nxcg.from_networkx(Gnx)

In [ ]:
Gcg = nxcg.from_networkx(Gnx)

In [ ]:
%%timeit
res = nx.betweenness_centrality(Gcg, k=50)

## What algorithms are supported?

|        Algorithm            |  Type         | Implemented |   Notes
| ----------------------------|---------------|-------------| ---------
| Betweeneess Centrality      | Centrality    |      X      | cuGraph 23.10 nightly         
| Edge Betweenness Centrality | Centrality    |      X      | cuGraph 23.10 nightly
| Louvain                     | Community     |      X      | cuGraph 23.10 nightly

## Future plans

We want to get your feedback! As this project develops, we want you to voice your opinions on which algorithms we should prioritize.